Load covid cases per population group from CDC (https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data/vbim-akqf)
The data list all individual cases (deidentified), with information on sex, age group, race and identity, and date of case, plus some additional information (hospitalisation, intensive care, death ...).
As the dataset is huge (20.6 Mrows at mid-march), and since their are many identical rows (because of deidentification), we query the data through the api, grouped by the categories we're interested in. Thus, the downloaded dataset remains relatively small.

In [30]:
!pip install sodapy

In [3]:
import pandas as pd
from sodapy import Socrata
import configparser
import json
import os

Without an application token, the size of the downloaded data is limited (see https://dev.socrata.com/docs/app-tokens.html). 

CDC uses the [Socrata Open Data API](https://dev.socrata.com/) to manage access to its data. To increase download capabilities, you have to create an AppToken.

First, [create a Socrata account](https://data.cdc.gov/signup). 

Then, [sign in](https://data.cdc.gov/login) to Socrata, using the Socrata ID field. Go to 'My Profile', then 'Edit profile', then tab 'Developer settings' (https://data.cdc.gov/profile/edit/developer_settings). Create an AppToken by following [this guide](https://support.socrata.com/hc/en-us/articles/210138558-Generating-an-App-Token).

Then store the AppToken into the config file `capstone.cfg` :
```
[CDC]
APPTOKEN=<MyAppToken>
```


In [21]:
config = configparser.ConfigParser()
config.read("capstone.cfg")

['capstone.cfg']

In [23]:
# Connect to CDC using AppToken for Socrata API
# set timeout to 100, since the query of such large dataset may take longer time tat default timeout
app_token = config["CDC"]["APPTOKEN"]
client = Socrata("data.cdc.gov",
                 app_token,
                timeout = 100)

In [13]:
# identifier of CDC dataset with covid case surveillance (https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data/vbim-akqf)
cdc_dataset_identifier = "vbim-akqf"

Column names of the data set. We will use only sex, age, ethnicity and date.

In [14]:
metadata = client.get_metadata(cdc_dataset_identifier)
[x['name'] for x in metadata['columns']]

['cdc_case_earliest_dt ',
 'cdc_report_dt',
 'pos_spec_dt',
 'onset_dt',
 'current_status',
 'sex',
 'age_group',
 'race_ethnicity_combined',
 'hosp_yn',
 'icu_yn',
 'death_yn',
 'medcond_yn']

documentation about Socrata API is available [here](https://dev.socrata.com/docs/queries/). Some exemple uses of the python client library  [sodapy](https://github.com/xmunoz/sodapy) can be found [here](https://github.com/xmunoz/sodapy/blob/master/examples/soql_queries.ipynb)

In [24]:
# by default, max number of records returned is 1000. We have to set it to 200000 to get the full results
res_sex_age_race_date = client.get(cdc_dataset_identifier, 
                     group = "cdc_case_earliest_dt, sex, age_group, race_ethnicity_combined",
                     select = "cdc_case_earliest_dt, sex, age_group, race_ethnicity_combined, count(*)",
                                  limit = 200000)

In [26]:
# number of different groups, and corresponding total number of rows (must match the total number of rows of the whole file)
print(len(res_sex_age_race_date))
sum( [ int(row["count"]) for row in res_sex_age_race_date])

95735


22507139

In [27]:
# output it to json for further use
with open(os.path.join("OUT_DATA", "covid_by_pop_group.json"), "w") as fs :
    json.dump(res_sex_age_race_date, fs, indent = 2)

In [28]:
res_sex_age_ethnicity_death =client.get(cdc_dataset_identifier, 
                     group = "sex, age_group, race_ethnicity_combined, death_yn",
                     select = "sex, age_group, race_ethnicity_combined, death_yn, count(*)",
                                  limit = 200000)

In [29]:
print(len(res_sex_age_ethnicity_death))
sum( [ int(row["count"]) for row in res_sex_age_ethnicity_death])

1327


22507139

In [10]:
with open(os.path.join("OUT_DATA", "covid_death_by_pop_group.json"), "w") as fs :
    json.dump(res_sex_age_ethnicity_death, fs)
    

In [18]:
df_death = pd.DataFrame(res_sex_age_ethnicity_death)
df_death = df_death.astype({"count" : "int"})

In [19]:
df_death.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   sex                      1320 non-null   object
 1   age_group                1320 non-null   object
 2   race_ethnicity_combined  1320 non-null   object
 3   death_yn                 1320 non-null   object
 4   count                    1320 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 51.7+ KB


In [35]:
df_death.groupby(["sex", "death_yn"]).sum()

count
sex     death_yn         
Female  Missing   3846666
        No        5489794
        Unknown   1114997
        Yes        169984
Male    Missing   3534902
        No        4939349
        Unknown   1019183
        Yes        201449
Missing Missing     35643
        No          12761
        Unknown      2051
        Yes           276
NA      Missing         6
        No              9
Other   Missing       164
        No            225
        Unknown        14
        Yes             6
Unknown Missing    108058
        No          75437
        Unknown     13508
        Yes           863

In [34]:
df_death.groupby(["sex", "death_yn"]).sum().loc[ (["Female", "Male"],["Yes", "No"]) ]

KeyError: "None of [Index(['Yes', 'No'], dtype='object')] are in the [columns]"